# R factor - basics


The [basic reproduction number (Wikipedia)](https://en.wikipedia.org/wiki/Basic_reproduction_number) can be thought of as the expected number of cases directly generated by one case in a population where all individuals are susceptible to infection. 

One way to represent $R$ is $R = \beta \,\tau $ where $\beta$ represents an average infection-producing contacts per unit time and $\tau$ the infectious period.

For exponential growth with case numbers N(t) increasing as $N(t) = N(t_0) R^{t/\tau}$, the _logarithmic growth rate K_ can be used to describe the growth: $K = \frac{d ln(N)}{dt}$.

The relationship between $R$ and $K$ is $R = \exp(K\tau)$ or equivalently $K = \ln(R)/\tau$.

[1] [Basic reproduction number (Wikipedia)](https://en.wikipedia.org/wiki/Basic_reproduction_number)

## Numerical exploration of the very basics

In [ ]:
%config InlineBackend.figure_formats = ['svg']
import numpy as np

In [ ]:
n_points = 15
t = np.arange(0, n_points, 1)
# tdiff = np.arange(0.5, n_points - 1, 1)
N0 = 1              # infections on day 0
tau = 4             # average infectious time
R0 = 2.1            # R0 - number of infections per period tau
n = N0*(R0**(t/tau))   # compute vector n with perfect exponential growth

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.plot(t, n, 'o', label=f'n = {N0} * ({R0} ^ (t/{tau}))')
ax.legend()
ax.set_xlabel("time [days]");
ax.set_ylabel("n(t)");

## Compute K and R0 from this data

In [ ]:
ln_n = np.log(n)

In [ ]:
fig, ax = plt.subplots()
K = np.diff(ln_n)
ax.plot(t, ln_n, 'o-', label='ln(n)')
ax.plot(tdiff, K, 'x-', label='K = d ln(n) / dt')
ax.legend()

In [ ]:
R0_reconstructed = np.exp(K*tau)

In [ ]:
R0_reconstructed

In [ ]:
assert R0_reconstructed[0] == R0

## Compute R0 from measured data

### Method 1: 

The bulletin from the Robert Koch institute [2] reports that an average infectious period of $\tau = 4$ days is assumed. Based on that information, the description of the method to compute $R$ (or $R_0$) is [3]

- compute an average $<n>_1$ of daily new infections over 4 days (say days 0 to 3)
- compute an average $<n>_2$ of daily new infections over 4 subsequent days (say days 4 to 7)
- compute the quotient $<n>_2 / <n>_1$ 

The method is references as being reported in [4].

[2] [Robert Koch Institute: Epidemiologisches Bulletin 17 | 2020 23. April 2020, page 13 onwards](https://www.rki.de/DE/Content/Infekt/EpidBull/Archiv/2020/Ausgaben/17_20.html)

[3] "_Bei einer konstanten Generationszeit von 4 Tagen, ergibt sich R als Quotient der Anzahl von Neuerkran- kungen in zwei aufeinander folgenden Zeitabschnitten von jeweils 4 Tagen. Der so ermittelte R-Wert wird dem letzten dieser 8 Tage zugeordnet, weil erst dann die gesamte Information vorhanden ist._"

[4] Wallinga J, Lipsitch M: How generation intervals shape the relationship between growth rates and reproductive numbers. [Proceedings of the Royal Society B:
Biological Sciences 2007;274(1609):599–60](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1766383/)

We test this here:

In [ ]:
import pandas as pd

In [ ]:
s = pd.Series(n, index=t)    # turn numpy array into pandas Series

In [ ]:
c = s.diff().dropna()        # compute the change from day to day 
                             # (i.e. dn/dt, time unit is one day
                             # and drop the first value for which can't compute dn/dt
                             # and for which the `diff` method has insert NaN

In [ ]:
mean1 = c[0:4].mean()        # take mean over first 4 days
mean2 = c[4:8].mean()        # take mean over next 4 days

In [ ]:
quotient = mean2 / mean1     # Compute R as quotient of the two means
quotient

In [ ]:
assert quotient == R0

Seems to be okay (for perfect exponential growth)

### method 2: K from d log(n)/dt and tau

In [ ]:
K = np.log(s).diff().dropna()

R0_reconstructed2 = np.exp(K*tau)
R0_reconstructed2

In [ ]:
assert R0_reconstructed2.values[0] == R0

## More complicated example with time-dependent R0

Now we'll use the same equations on synthetic data, to see if we can re-construct the underlying R0.

### Step changes in R0

In [ ]:
n_points = 50
t = np.arange(0, n_points, 1)
N0 = 3
tau = 4
# R0_td = 3 - 0.1*(np.sin(t/n_points*2*np.pi*2))   
R0_td = np.zeros(shape=t.shape)
R0_td[:n_points//3] = 3
R0_td[n_points//3:2*n_points//3] = 2
R0_td[2*n_points//3:] = 2.5

# R0_td = 2.0 - 0.5*(t/n_points)
R0 = 2.0
# n_td = N0*(R0_td**(t/tau))   # inaccurate, need fake_growth for this
n = N0*(R0**(t/tau))

In [ ]:
def fake_growth(t, R, tau: float, N0=1):
    """expect 
    - t: a vector (counting days)
    - Rs: a vector with the desired R for each day
    - tau: a constant for the assumption of the average infectious period
    
    Assumes exponential growth according to
    N[i] = N0*(R**(t/tau)) from day i to day i+1 with rate R[i].
    
    and compute the increase from day i to day i by computing
    dN[i] = N[i+1] - N[i]
    
    Compute a time series n, so that this accumulates dN. Return n.
    """
    def f(R_, t_, tau, N0):
        return N0 * R_**(t_/tau)
    
    dN = np.zeros(shape=t.shape)
    n = np.zeros(shape=t.shape)
    assert len(t) == len(R)
    assert t[0] == 0
    dN[0] = N0
    for i in range(1, len(t)):
        # if R < 1, might get negative dN. Let's not allow this for now.
        assert R[i] >= 1, f"R[{i}] = {R[i]} <= 1.0"
        N_i = f(R[i], t[i], tau, N0)
        N_im1 = f(R[i], t[i-1], tau, N0)   # read N_i_minus_1
        dN_i = N_i - N_im1
        # import IPython
        # IPython.embed()
        dN[i] = dN_i
        if dN_i < 0:
            print(f"dN[{i}] = {dN[i]}, N[{i}] = {N_i}, N[{i-1}] = {N_im1}, R[{i}] = {R[i]}")
            # IPython.embed()
    
    # Compute accumulated cases
    # for day 0, we expect N0 (assuming that t[0] == 0)
    assert t[0] == 0
    n[0] = N0
    for i in range(1, len(t)):
        n[i] = n[i-1] + dN[i]
    return n

def test_fake_growth():
    """ Assumer constant growth rate, and expect exponential growth."""
    npoints = 20
    t = np.arange(0, npoints, 1)
    R_number = 2.1
    R = np.ones(shape=t.shape) * R_number
    tau = 3.5
    N0 = 4
    n1 = fake_growth(t, R, tau, N0)
    n2 = N0 * R_number ** (t / tau)
    diff = n2-n1
    max_err = np.max(np.abs(diff))
    print(f"Max error is {max_err}")
    assert max_err < 1e-15
    return n1, n2

test_fake_growth()

In [ ]:
n_fake = fake_growth(t, R0_td, tau)

In [ ]:
n_fake = pd.Series(n_fake)

In [ ]:
import matplotlib.pyplot as plt
fig, (ax, ax2, ax3) = plt.subplots(3, 1)
# ax.plot(t, n, 'o', label=f'n = {N0} * ({R0} ^ (t/{tau}))')
# ax.plot(t, n_td, 'o', color='C1', label=f'n = {N0} * (R0(t) ^ (t/{tau}))')
ax.step(t, n_fake, 'o-', color='C3', label=f'n = fake growth)')

ax3.plot(t, R0_td, '-o',color='C1', label="time dependent R0")

ax2.bar(t, n_fake.diff(), label='daily new cases')

ax3.set_xlabel('time [days]');
ax.legend(), 
ax2.legend()

In [ ]:
df = pd.DataFrame({'R_td' : R0_td, 'n_td' : n_fake, 'c_td' : n_fake.diff()})

## Method 1: RKI

In [ ]:
# Smoothing makes things a lot better:
df['smooth_c'] = df['c_td'].rolling(7, center=True, 
                                    win_type='gaussian', min_periods=7).mean(std=3)
# but important to have min_periods the same as the rolling period

#df['mean4d'] = df['c_td'].rolling(4).mean()
df['mean4d'] = df['smooth_c'].rolling(4).mean()

df['R_td-recon'] = df['mean4d'] / df['mean4d'].shift(4)

df[['R_td', 'R_td-recon', 'n_td', 'mean4d']].head(n=20)

In [ ]:
# df[['R_td', 'R_td-recon', 'n_td', 'mean4d']].tail(n=4)
df[['R_td', 'R_td-recon']].tail(n=4)

In [ ]:
fig, ax = plt.subplots()
ax.plot(t, df['R_td'], '-x', label='R_td')
ax.plot(t, df['R_td-recon'], 'o', label=f'n = R_td-recon')
ax.legend()
ax.set_title('Method 1');

## Method 2

In [ ]:
df['K'] = np.log(df['n_td']).diff(periods=1)

df['R_td-recon2'] = np.exp(df['K'] * tau)


In [ ]:
df[['R_td', 'R_td-recon2']].head()

In [ ]:
df[['R_td', 'R_td-recon2']].tail()

In [ ]:
fig, ax = plt.subplots()
ax.plot(t, df['R_td'], '-x', label='R_td')
ax.plot(t, df['R_td-recon2'], 'o', color='C2', label=f'R_td-recon2')
ax.legend()
ax.set_title('Method 2');

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
ax.plot(t, df['R_td'], '-x', linewidth=3, label='R_td')
ax.plot(t, df['R_td-recon'], 'o:', color="C1", label=f'n = R_td-recon')
ax.plot(t, df['R_td-recon2'], 'o:', color="C2", label=f'n = R_td-recon2')
ax.legend()
ax.set_title('Method 1 and 2 in comparison');
ax.grid('on')

### Test 2: random noise

In [ ]:
n_points = 50
t = np.arange(0, n_points, 1)
N0 = 3
tau = 4
R0_td = np.zeros(shape=t.shape) + 2   
noise = np.random.uniform(size=t.shape) - 0.5
# add 10% noise (relative error to actual signal)
n_fake = pd.Series(fake_growth(t, R0_td, tau) * (1 + 0.1 * noise))



In [ ]:

fig, (ax, ax2, ax3) = plt.subplots(3, 1)
ax.step(t, n_fake, 'o-', color='C3', label=f'n = fake growth)')
ax3.plot(t, R0_td, '-o',color='C1', label="time dependent R0")
ax2.bar(t, n_fake.diff(), label='daily new cases')

ax3.set_xlabel('time [days]');
ax.legend(), 
ax2.legend()

In [ ]:
df = pd.DataFrame({'R_td' : R0_td, 'n_td' : n_fake, 'c_td' : n_fake.diff()})

### Method 1: RKI

In [ ]:
df['mean4d'] = df['c_td'].rolling(4).mean()
df['R_td-recon'] = df['mean4d'] / df['mean4d'].shift(4)

df[['R_td', 'R_td-recon', 'n_td', 'mean4d']].head(n=20)

In [ ]:
fig, ax = plt.subplots()
ax.plot(t, df['R_td'], '-x', label='R_td')
ax.plot(t, df['R_td-recon'], 'o', label=f'n = R_td-recon')
ax.legend()
ax.set_title('Method 1');

The noise seems amplified in the reconstructed R.

Let's try some smoothing of the noisy data:

In [ ]:

df['smooth_c'] = df['c_td'].rolling(7, center=True, 
                                    win_type='gaussian', min_periods=7).mean(std=3)
df['mean4d'] = df['smooth_c'].rolling(4).mean()


df['R_td-recon'] = df['mean4d'] / df['mean4d'].shift(4)

df[['R_td', 'R_td-recon', 'n_td', 'mean4d']].head(n=20)

In [ ]:
fig, ax = plt.subplots()
ax.plot(t, df['R_td'], '-x', label='R_td')
ax.plot(t, df['R_td-recon'], 'o', label=f'n = R_td-recon')
ax.legend()
ax.set_title('Method 1');

## Conclusions

1. Use RKI algorithm: seems stable, returns the correct value R (after ~tau days) if R is constant

2. Use smoothing for the case numbers (or the diff of the case numbers) to improve estimates.

3. It is important for the rolling averages (both for the smoothing of the diff, and for the
4-day average) to use all data points, and not to ignore some. If we use 'min_values=' to allow fewer data points, the reconstructed R values show systematic errors at the beginning and end of the interval. 

(Maybe that can be improved.)

Draft algorithm here:

In [ ]:
def compute_R(daily_change, tau=4):
    """Given a time series s, estimate R based on description from RKI [1].
    
    [1] [Robert Koch Institute: Epidemiologisches Bulletin 17 | 2020 23. April 2020]
    https://www.rki.de/DE/Content/Infekt/EpidBull/Archiv/2020/Ausgaben/17_20.html

    Steps: 
    
    1. Compute change from day to day
    2. Take tau-day averages (tau=4 is recommended as of April/May 2020)
    3. divide average from days 4 to 7 by averages from day 0 to 3, and use this data point for day[7]

    """
    # change = s.diff()
    change = daily_change
    mean4d = change.rolling(tau).mean()
    R = mean4d / mean4d.shift(tau)
    R2 = R.shift(-tau)  # this is not the RKI method, but seems more appropriate.
    return R2
    

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
ax.plot(t, df['R_td'], '-x', linewidth=3, label='R_td')

ax.plot(t, compute_R(df['n_td'].diff()), 'o:', color="C1", label=f'n = R_td-recon')
ax.legend()
ax.grid('on')

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
ax.plot(t, df['R_td'], '-x', linewidth=3, label='R_td')
ax.plot(t, compute_R(df['n_td'].diff()), 'o:', color="C1", label=f'n = R_td-recon')
ax.legend()
ax.grid('on')

In [ ]:
import oscovida
from oscovida import compute_growth_factor

In [ ]:
cases, deaths = oscovida.get_country_data("Germany")


In [ ]:
import math
def min_max_in_past_n_days(series, n, at_least = [0.75, 1.25], alert=[0.5, 100]):
    """Given a time series, find the min and max values in the time series within the last n days.
    
    If those values are within the interval `at_least`, then use the values in at_least as the limits.
    if those values are outside the interval `at_least` then exchange the interval accordingly.
    
    If the values exceed the min and max value in 'alerts', then print an error message.
    Return min, max.
    """
    if n > len(series):
        n = len(series)
        
    series = series.replace(math.inf, math.nan)
        
    min_ = series[-n:].min()
    max_ = series[-n:].max()
    
    if min_ < at_least[0]:
        min_final = min_
    else:
        min_final = at_least[0]
        
    if max_ > at_least[1]:
        max_final = max_
    else:
        max_final = at_least[1]
        
    if max_final > alert[1]:
        print(f"Large value for R_max = {max_final} > {alert[1]} in last {n} days: \n", series[-n:])
    if min_final < alert[0]:
        print(f"Small value for R_min = {min_final} < {alert[0]} in last {n} days: \n", series[-n:])
        
    print(f"DDD: min_={min_}, max_={max_}")
    return min_final, max_final

In [ ]:


cases, deaths = oscovida.get_country_data("Iran")

fig, ax = plt.subplots(1, 1 , figsize=(12, 4))
# change, smooth, smooth2 = oscovida.compute_daily_change(cases)
oscovida.plot_growth_factor(ax, cases, "C1")
# oscovida.plot_growth_factor(ax, deaths, "C0")
diff = cases.diff()
smooth_diff = diff.rolling(9, center=True, win_type='gaussian').mean(std=4)

R = compute_R(smooth_diff)
ax.plot(R.index, R, "-C4", label=r"R (estimated with $\tau$=4 days using RKI algorithm)", linewidth=3)

ax.legend()
min_, max_ = min_max_in_past_n_days(R, 28);
ax.plot([R.index.min(), R.index.max()], [min_, min_], 'b-')
ax.plot([R.index.min(), R.index.max()], [max_, max_], 'b-')
ax.set_ylim([min_, max_]);

In [ ]:
pd.set_option('max_rows', None)

In [ ]:
R[-21:].min()

In [ ]:
ax.plot(R.index, R, "C2", label="R")

In [ ]:
oscovida.overview("Iran");

In [ ]:
def plot_reproducion_number(ax, series, color):
    """Documentation to be added
    """

   
    # data for computation or R
    # change, smooth, smooth2 = oscovida.compute_daily_change(series)
    
    smooth = series.diff().rolling(7, center=True, win_type='gaussian').mean(std=4)
    
    R = compute_R(smooth)
    ax.plot(R.index, R, "-C4", label=r"estimated R (assume $\tau$=4 days, using RKI algorithm)", linewidth=3)

    # choose y limits so that all data points of R in the last 28 days are visible
    min_, max_ = min_max_in_past_n_days(R, 28);
    ax.set_ylim([min_, max_]);

    # Plot ylim interval for debugging
    # ax.plot([R.index.min(), R.index.max()], [min_, min_], 'b-')
    # ax.plot([R.index.min(), R.index.max()], [max_, max_], 'b-')
    
     # get smooth data for growth factor from plot 1 to base this plot on
    (f, f_label) , (f_smoothed, smoothed_label) = compute_growth_factor(series)
    
    label = series.country + " " + series.label + " daily growth factor " + f_label
    ax.plot(f.index, f.values, 'o', color=color, alpha=0.3, label=label)

    label = series.country + " " + series.label + " daily growth factor " + smoothed_label
    ax.plot(f_smoothed.index, f_smoothed.values, '-', color=color, label=label, linewidth=LW)

    ax.set_ylabel("R and daily growth factor")
    # plot line at 0
    ax.plot([series.index.min(), series.index.max()], [1.0, 1.0], '-k') # label="critical value"
    ax.legend()
    return ax


In [ ]:
fig, ax = plt.subplots(1, 1 , figsize=(12, 4))
LW = oscovida.LW
plot_reproducion_number(ax, cases, 'C1')
LW = oscovida.LW